In [ ]:
import os
import requests

os.environ['http_proxy'] = "http://127.0.0.1:7891"
os.environ['https_proxy'] = "http://127.0.0.1:7891"
# 测试，输出200为正常接通
response = requests.get('http://www.google.com')
print(response.status_code)


In [ ]:
%cd /data/ypq/DiffusionPainter/DiffCut

In [ ]:
from diffcut.ldm import LdmExtractor

In [ ]:
def mask_refinement(labels, image):
    masks = torch.cat([1. * (labels == label) for label in torch.unique(labels)], dim=1)
    # num_iter and dilations impact final result
    labels = PAMR(num_iter=30, dilations=[1, 2, 4, 8, 12, 24, 32])(image, masks)

    labels = 1. * torch.argmax(labels, dim=1)
    labels = median_filter(labels.cpu().numpy(), 3).astype(int)

    return labels

#### __Load Diffusion Model__

In [ ]:
extractor = LdmExtractor(model_name="SSD-1B", inverse_scheduler=None)

#### __Diffusion Features Extraction__

In [ ]:
from utils_diffcut import *
img_path = "./assets/iceland.png"

pil_img = Image.open(img_path).convert('RGB')
image = T.ToTensor()(pil_img).unsqueeze(0).to("cuda")
_, _, h, w = image.shape

img_size = 1024  # 2048

resized_image = F.interpolate(image, size=(img_size, img_size), mode='bilinear')
batched_input = {'img': torch.cat([resized_image], dim=0), 'caption': [""]}

# Diffusion Timesteps
t = 50
diffusion_features = extractor(batched_input, guidance_scale=1., n_steps=10, steps=(t,), encode_method="noise")

# Output features from the final self-attention block of the UNet encoder
features = diffusion_features['down_blocks.2.attentions.1.transformer_blocks.3']  # MODULE INDEX STARTING FROM '0'
print(features.shape)

#### __Zero-Shot Image Segmentation__

In [ ]:
# Hyperparameters
tau = 0.5
alpha = 10


# Recursive Feature Clustering
feats = features.reshape(1, img_size//32, img_size//32, -1).permute(0, 3, 1, 2).to(torch.float32)
feats = F.normalize(feats, dim=1)
diffcut  = DiffCut()


clusters, feats_norm = diffcut.generate_clusters(feats, tau, alpha)
cluster_mask = diffcut.label_association(clusters[0], feats_norm, org_size=(h, w))
cluster_mask = torch.Tensor(cluster_mask).to("cuda")
print(cluster_mask.shape)

In [ ]:
# Mask Refinement
# TODO: img_size is the needed canvas size
image = F.interpolate(image, size=(img_size, img_size), mode='bilinear')
refined_single_mask = single_mask_refinement(cluster_mask, image)

plt.figure(figsize=(6, 6))
plt.imshow(refined_single_mask, cmap='gray')  # Use gray colormap for binary mask visualization
plt.title('Refined Mask')
plt.axis('off')  # Hide axes for better visual representation
plt.show()
